In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull(),cmap = 'viridis')

In [ ]:
df['rating'].value_counts().unique()

**Dropping director & cast columns as they are not playing important part in analysis**

In [ ]:
df.drop(['director','cast'],axis=1,inplace=True)

In [ ]:
df.head()                 #after dropping columns

.**We replaced all the Nan values in the country column with United States as Netflix was created in the USA and every show is aired on Netflix US. So instead of dropping the whole column we just replaced the values in it in order to save our data.**

In [ ]:
df['country'].replace(np.nan, 'United States',inplace=True)

**We already have released year for each movie and hence even if we dont have released date,it wont affect our analsis much. Hence we can Drop released date column**

In [ ]:
df.drop(['date_added'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['rating'].value_counts()

In [ ]:
df['listed_in'].value_counts()

**As we can see we only have 10 missing values in our rating column, we can either drop them or replace them. We have TV-MA which is the most common raing and hence we can replace all these nan values with TV-MA.**

In [ ]:
df['rating'].replace(np.nan,'TV-MA',inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
sns.countplot(x='type',data=df)

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='rating',data=df)

In [ ]:
plt.figure(figsize = (35,6))
sns.countplot(x='release_year',data = df)

In [ ]:
plt.figure(figsize=(16,6))
sns.scatterplot(x='rating',y='type',data=df)

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='rating',data=df,hue='type')

In [ ]:
#distribution according to countries
import plotly.express as px
top_rated=df[0:10]
fig=px.sunburst(top_rated,path=['country'])
fig.show()

In [ ]:
#distribution according to the rating
df['rating'].value_counts().plot.pie(autopct='%1.1f%%',figsize=(20,35))
plt.show()

In [ ]:
 #oldest movies available on netflix
old =df.sort_values("release_year",ascending=True)
old=old[old['duration']!= "" ]
old[['title','release_year']][:15]

**standup shows on Netflix**

In [ ]:
tag = "Stand-Up Comedy" #standup shows on Netflix
df["relevant"] = df['listed_in'].fillna("").apply(lambda x : 1 if tag.lower() in x.lower() else 0)
com = df[df["relevant"] == 1]
com[com["country"] == "United States"][["title", "country","release_year"]].head(10)

In [ ]:
 #Kids TV shows on Netflix
tag= "Kids' TV"
df["relevant"]=df['listed_in'].fillna("").apply(lambda x:1 if tag.lower() in x.lower() else 0)
com=df[df["relevant"]==1]
com[com["country"] == "United States"][["title", "country","release_year"]].head(10)

In [ ]:
df_countries = pd.DataFrame(df.country.value_counts().reset_index().values, columns=["country", "count"])
df_countries.head()

In [ ]:
#distribution of content on basis of countries
fig = px.choropleth(
locationmode='country names',
locations =df_countries["count"])
fig.show()

In [ ]:
date=pd.DataFrame(df.release_year.value_counts().reset_index().values,columns=["Year","Count"])
date.head()

In [ ]:
plt.figure(figsize=(12,6))
df[df["type"]=="Movie"]["release_year"].value_counts()[:20].plot(kind="bar",color="Red")
plt.title("Frequency of Movies which were released in different years and are available on Netflix")

In [ ]:
plt.figure(figsize=(12,6))
df[df["type"]=="TV Show"]["release_year"].value_counts()[:20].plot(kind="bar",color="Blue")
plt.title("Frequency of TV shows which were released in different years and are available on Netflix")

In [ ]:
plt.figure(figsize=(12,6))
df[df["type"]=="Movie"]["listed_in"].value_counts()[:10].plot(kind="barh",color="black")
plt.title("Top 10 Genres of Movies",size=18)

In [ ]:
plt.figure(figsize=(12,6))
df[df["type"]=="TV Show"]["listed_in"].value_counts()[:10].plot(kind="barh",color="brown")
plt.title("Top 10 Genres of TV Shows",size=18)

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.subplots(figsize=(25,15))
wordcloud = WordCloud(
                          background_color='Black',
                          width=1920,
                          height=1080
                         ).generate(" ".join(df.title))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('cast.png')
plt.show()